In [2]:
# Model definition
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from keras import models
from keras import layers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor

from keras.preprocessing.image import (ImageDataGenerator,)
from keras.datasets import (boston_housing,mnist)
from keras.utils import to_categorical

from sklearn.preprocessing  import ( StandardScaler,)
from sklearn.model_selection import ( train_test_split,KFold, )

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
train_dir = "G:/KAGGLE/Cats And Dogs/train"
valid_dir = "G:/KAGGLE/Cats And Dogs/valid"

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150), batch_size = 20, class_mode= 'binary') 
valid_generator = test_datagen.flow_from_directory(valid_dir, target_size = (150,150), batch_size = 20, class_mode= 'binary') 

Found 24800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation= 'relu'))
model.add(layers.Dense(1, activation= 'sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 15, 15, 128)       147584    
__________

In [15]:
model.compile(optimizer = optimizers.RMSprop(lr=1e-4), loss = 'binary_crossentropy', metrics = ['acc'])
#model.fit(train_images, train_labels, epochs=5, batch_size=1000)

model

In [ ]:
epochs = 2
history = model.fit_generator(train_generator, steps_per_epoch = 10, epochs = epochs, validation_data = valid_generator, validation_steps = 50)

Epoch 1/2
10/10 [==============================] - 13s 1s/step - loss: 0.6755 - acc: 0.6000 - val_loss: 0.6256 - val_acc: 0.6900
Epoch 2/2
 6/10 [=================>............] - ETA: 2s - loss: 0.6544 - acc: 0.6500

In [18]:
plt.plot(range(0,epochs),history.history["acc"], label = "train acc")
plt.plot(range(0,epochs),history.history["val_acc"],label = "valid  acc")
#plt.plot(epochs,history.history["mae"],label = "mae")
plt.legend()
plt.xlabel("epochs")
plt.ylabel("accuracy")

plt.plot(range(0,epochs),history.history["loss"], label = "train loss")
plt.plot(range(0,epochs),history.history["val_loss"],label = "valid  loss")
#plt.plot(epochs,history.history["mae"],label = "mae")
plt.legend()
plt.xlabel("epochs")
plt.ylabel("loss")

NameError: name 'history' is not defined